<a href="https://colab.research.google.com/github/zideric/colab/blob/main/ConvNet_con_Keras_(e_Pooling).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ConvNet con Keras

creaimo una rete neurale convoluzionale con Keras. usiamo sempre il Fashion-MNIST. al termine vedremo quanto con la convoluzione il metodo è migliore rispetto alla semplice rete neurale che abbiamo creato nei precedenti Notebook

importiamo i moduli

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense

from keras.utils import to_categorical

carichiamo il fashion-MNIST all'interno di due array numpy

In [2]:
from keras.datasets import fashion_mnist

labels = ["T-shirt/top","Pantalone","Pullover","Vestito","Cappotto","Sandalo","Maglietta","Sneaker","Borsa","Stivaletto"]

(X_train, y_train) ,(X_test, y_test) = fashion_mnist.load_data()

print("Numero totale di proprietà: "+str(X_train.shape[1]))
print("Esempi di training: "+str(X_train.shape[0]))
print("Esempi di test: "+str(X_test.shape[0]))

4423680/4422102 [==============================] - 0s 0us/step
Numero totale di proprietà: 28
Esempi di training: 60000
Esempi di test: 10000


normalizziamo i dati


In [3]:
X_train = X_train/255
X_test = X_test/255

L'input di una rete convoluzionale deve avere dimensione del tipo NHW*C, dove:

* N è il numero di esempi nel dataset
* H e W sono rispettivamente altezza e larghezza dell'immagine in pixel
* C sono i canali dell'immagine

Nel caso di un'immagine a colori (formato RGB) i canali saranno 3, per un immagine in bianco e nero invece abbiamo un solo canale. Utilizziamo il metodo reshape per modificare la dimensione degli array con le features, in modo da conformarci alle specifiche di Keras.

In [4]:
print(X_train.shape[0])
print(X_train.shape[1])
print(X_train.shape[2])

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1],X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1],X_test.shape[2], 1)
X_train.shape

60000
28
28


(60000, 28, 28, 1)

creiamo le variabili di comodo per il target

In [5]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

## Creiamo il modello

In [6]:
from keras.layers import Conv2D, Flatten, Dropout

model = Sequential()

model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 64)        320       
_________________________________________________________________
flatten (Flatten)            (None, 50176)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               6422656   
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 6,424,266
Trainable params: 6,424,266
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [8]:
model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=10)

Epoch 1/10
94/94 [==============================] - 34s 23ms/step - loss: 0.9616 - accuracy: 0.6632 - val_loss: 0.3850 - val_accuracy: 0.8589
Epoch 2/10
94/94 [==============================] - 2s 18ms/step - loss: 0.4198 - accuracy: 0.8551 - val_loss: 0.3230 - val_accuracy: 0.8825
Epoch 3/10
94/94 [==============================] - 2s 18ms/step - loss: 0.3511 - accuracy: 0.8780 - val_loss: 0.2972 - val_accuracy: 0.8898
Epoch 4/10
94/94 [==============================] - 2s 18ms/step - loss: 0.3082 - accuracy: 0.8903 - val_loss: 0.2824 - val_accuracy: 0.8976
Epoch 5/10
94/94 [==============================] - 2s 18ms/step - loss: 0.2750 - accuracy: 0.9030 - val_loss: 0.2704 - val_accuracy: 0.9014
Epoch 6/10
94/94 [==============================] - 2s 18ms/step - loss: 0.2517 - accuracy: 0.9098 - val_loss: 0.2626 - val_accuracy: 0.9049
Epoch 7/10
94/94 [==============================] - 2s 18ms/step - loss: 0.2379 - accuracy: 0.9141 - val_loss: 0.2611 - val_accuracy: 0.9051
Epoch 8/10
9

vediamo sul test set


In [9]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.2774 - accuracy: 0.9011


[0.27740466594696045, 0.9010999798774719]

## Utilizzare uno strato di Pooling

lo strato di pooling semplifica la matrice e il modello riducendo l'overfitting 

In [22]:
from keras.layers import MaxPooling2D

model = Sequential()

model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=2,strides=2))
model.add(Dropout(0.3))
model.add(Conv2D(filters=32, kernel_size=2,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2,strides=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 28, 28, 64)        320       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 14, 32)        8224      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1568)             

In [23]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

introduciamo l'early stopping, server per fermare l'elaborazione se per un numero X di epoche non ho ottenuto un miglioramento di almeno Y, posso anche decidere se utilizzare l'ultimo step o lo step migliore eventuale di epoche precedenti

In [24]:
from keras.callbacks import History,EarlyStopping

history = History()
earlyStopping = EarlyStopping(min_delta=0.001, patience=5)

model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=100, callbacks=[history, earlyStopping])


Epoch 1/100
94/94 [==============================] - 2s 21ms/step - loss: 1.2544 - accuracy: 0.5492 - val_loss: 0.5229 - val_accuracy: 0.8069
Epoch 2/100
94/94 [==============================] - 2s 19ms/step - loss: 0.5309 - accuracy: 0.8037 - val_loss: 0.4258 - val_accuracy: 0.8481
Epoch 3/100
94/94 [==============================] - 2s 19ms/step - loss: 0.4535 - accuracy: 0.8342 - val_loss: 0.3788 - val_accuracy: 0.8660
Epoch 4/100
94/94 [==============================] - 2s 19ms/step - loss: 0.4078 - accuracy: 0.8515 - val_loss: 0.3483 - val_accuracy: 0.8737
Epoch 5/100
94/94 [==============================] - 2s 19ms/step - loss: 0.3815 - accuracy: 0.8601 - val_loss: 0.3319 - val_accuracy: 0.8806
Epoch 6/100
94/94 [==============================] - 2s 19ms/step - loss: 0.3587 - accuracy: 0.8696 - val_loss: 0.3211 - val_accuracy: 0.8842
Epoch 7/100
94/94 [==============================] - 2s 19ms/step - loss: 0.3403 - accuracy: 0.8772 - val_loss: 0.3091 - val_accuracy: 0.8884
Epoch 

In [25]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.2313 - accuracy: 0.9172


[0.23126330971717834, 0.9172000288963318]